<a href="https://colab.research.google.com/github/Only-Mike/DeepLearning-and-AI/blob/main/Assignment4_Question_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load (some of) required libraries
!pip install -U transformers huggingface_hub datasets evaluate -qq
from datasets import load_dataset
import pandas as pd
import numpy as np



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Login to Hugging Face Hub to be able to save the model
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
#dataset = load_dataset('Rami/adhd_question')

In [ ]:
#Make dataset into pandas dataframe
#train_df = pd.DataFrame(dataset['train'])
#test_df = pd.DataFrame(dataset['test'])

# Merge the dataset to make a new train/test split because the testset is bigger than train set
#df = pd.concat([train_df, test_df])


In [ ]:
#Load a part of the squad dataset for testing
squad = load_dataset("squad", split="train[:5000]")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


In [ ]:
# Split dataset into train/test with test size of 20%
squad = squad.train_test_split(test_size=0.2)

In [ ]:
squad["train"][0]

{'id': '56d10a4717492d1400aab7d8',
 'title': 'New_York_City',
 'context': "New York City has more than 2,000 arts and cultural organizations and more than 500 art galleries of all sizes. The city government funds the arts with a larger annual budget than the National Endowment for the Arts. Wealthy business magnates in the 19th century built a network of major cultural institutions, such as the famed Carnegie Hall and The Metropolitan Museum of Art, that would become internationally established. The advent of electric lighting led to elaborate theater productions, and in the 1880s, New York City theaters on Broadway and along 42nd Street began featuring a new stage form that became known as the Broadway musical. Strongly influenced by the city's immigrants, productions such as those of Harrigan and Hart, George M. Cohan, and others used song in narratives that often reflected themes of hope and ambition.",
 'question': 'What technological development led resulted in elaborate stage pro

In [ ]:
# Define the tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# Define a preprocess function to format our input data
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
# Tokenize the dataset using the defined tokenizer and preprocess function
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Define data collator
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
# Define the specific model we'll use
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Cloning https://huggingface.co/Only-Mike/my_awesome_qa_model into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/253M [00:00<?, ?B/s]

Download file runs/Mar09_13-41-12_e9000c53fa47/events.out.tfevents.1678369280.e9000c53fa47.242.0: 100%|#######…

Download file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

Clean file runs/Mar09_13-41-12_e9000c53fa47/events.out.tfevents.1678369280.e9000c53fa47.242.0:  25%|##4       …

...

Clean file pytorch_model.bin:   0%|          | 1.00k/253M [00:00<?, ?B/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 750
  Number of trainable parameters = 66364418


Epoch,Training Loss,Validation Loss
1,No log,2.321226
2,2.719400,1.934407
3,2.719400,1.852058


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to my_awesome_qa_model/checkpoint-500
Configuration saved in my_awesome_qa_model/checkpoint-500/config.json
Model weights saved in my_awesome_qa_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in my_awesome_qa_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in my_awesome_qa_model/checkpoint-500/special_tokens_map.json
tokenizer config file saved in my_awesome_qa_model/tokenizer_config.json
Special tokens file saved in my_awesome_qa_model/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=750, training_loss=2.323900146484375, metrics={'train_runtime': 516.9593, 'train_samples_per_second': 23.213, 'train_steps_per_second': 1.451, 'total_flos': 1175877900288000.0, 'train_loss': 2.323900146484375, 'epoch': 3.0})



---


# Created dataset from ChatGPT



---



---



In [ ]:
#Dataset created from ChatGPT
import random

questions = [
    "What is ADHD?",
    "What are the symptoms of ADHD?",
    "How is ADHD diagnosed?",
    "What are the treatments for ADHD?",
    "Can ADHD be cured?",
    "What is the difference between ADD and ADHD?",
    "Is ADHD hereditary?",
    "What are some strategies for managing ADHD?",
    "Can ADHD affect adults?",
    "How does ADHD affect learning?",
    "How does ADHD affect sleep?",
    "What are some common misconceptions about ADHD?",
    "What are some resources for people with ADHD?",
    "How can ADHD affect relationships?",
    "What are some ways to talk to your doctor about ADHD?",
]

contexts = [
    "Attention-deficit/hyperactivity disorder (ADHD) is a neurodevelopmental disorder characterized by inattention, hyperactivity, and impulsivity.",
    "The symptoms of ADHD include difficulty paying attention, hyperactivity, and impulsivity.",
    "ADHD is typically diagnosed through a combination of assessments, including a medical evaluation, a clinical interview, and rating scales completed by parents, teachers, or caregivers.",
    "Treatments for ADHD may include medication, therapy, or a combination of both.",
    "ADHD cannot be cured, but symptoms can be managed with appropriate treatment.",
    "ADD (attention-deficit disorder) is an outdated term that was previously used to describe individuals with inattention but without hyperactivity or impulsivity. ADHD is now used to describe all presentations of the disorder.",
    "There is evidence to suggest that ADHD has a genetic component, but the exact cause is not yet fully understood.",
    "Some strategies for managing ADHD include medication, therapy, exercise, and organizational tools such as calendars and to-do lists.",
    "ADHD can affect adults, and symptoms may manifest differently than in children.",
    "ADHD can affect learning by making it difficult to focus and pay attention in class.",
    "ADHD can affect sleep by causing difficulties falling asleep or staying asleep.",
    "Some common misconceptions about ADHD include that it only affects children, that it is caused by poor parenting, and that it can be cured with willpower or discipline.",
    "Some resources for people with ADHD include support groups, therapy, and accommodations at school or work.",
    "ADHD can affect relationships by causing difficulties with communication, organization, and emotional regulation.",
    "Some ways to talk to your doctor about ADHD include bringing up specific symptoms, discussing any concerns or questions, and asking about available treatments and resources."
]

answers = [
    "ADHD is a neurodevelopmental disorder characterized by inattention, hyperactivity, and impulsivity.",
    "The symptoms of ADHD can include difficulty paying attention, hyperactivity, and impulsivity, among others.",
    "ADHD is typically diagnosed through a combination of assessments, including a medical evaluation, a clinical interview, and rating scales completed by parents, teachers, or caregivers.",
    "Treatments for ADHD may include medication, therapy, or a combination of both, and the best approach may vary depending on individual needs.",
    "ADHD cannot be cured, but symptoms can be managed with appropriate treatment.",
    "ADD (attention-deficit disorder) is an outdated term that was previously used to describe individuals with inattention but without hyperactivity or impulsivity. ADHD is now used to describe all presentations of the disorder.",
    "There is evidence to suggest that ADHD has a genetic component, but the exact cause is not yet fully understood.",
    "Some strategies for managing ADHD include medication, therapy, exercise, and organizational tools such as calendars and to-do lists.",
    "ADHD can affect adults, and symptoms may manifest differently than in children. It is important for adults to seek proper diagnosis and treatment if they suspect they have ADHD.",
    "ADHD can affect learning by making it difficult to focus and pay attention in class. However, with appropriate accommodations and support, individuals with ADHD can still succeed academically.",
    "ADHD can affect sleep by causing difficulties falling asleep or staying asleep. It is important to address sleep issues in individuals with ADHD, as poor sleep can exacerbate symptoms.",
    "Some common misconceptions about ADHD include that it only affects children, that it is caused by poor parenting, and that it can be cured with willpower or discipline. These misconceptions can lead to stigmatization and misunderstanding of the disorder.",
    "Some resources for people with ADHD include support groups, therapy, and accommodations at school or work. It is important for individuals with ADHD to seek out appropriate support and resources.",
    "ADHD can affect relationships by causing difficulties with communication, organization, and emotional regulation. It is important for individuals with ADHD and their loved ones to understand the impact of the disorder on relationships and to seek out appropriate support.",
    "Some ways to talk to your doctor about ADHD include bringing up specific symptoms, discussing any concerns or questions, and asking about available treatments and resources."
    ]

# Set the number of examples you want to create
num_examples = 50

# Create an empty list to store the examples
examples = []

# Loop through the number of examples and create a new row for each one
for i in range(num_examples):
    # Choose a random question and context from the lists
    question = random.choice(questions)
    context = random.choice(contexts)
    
    # Create a new row for the example and add it to the list of examples
    examples.append({
        "id": i,
        "title": "ADHD",
        "context": context,
        "question": question,
        "answers": {"text": ["Answer not available"], "answer_start": [0]}
    })

# Convert the list of examples to a pandas dataframe
df = pd.DataFrame(examples)

# Reorder the columns to match the desired format
#df = 